In [1]:
from sklearn.cluster import KMeans # KMeans 라이브러리 불러오기
import numpy as np
import pandas as pd
import math
import seaborn as sb # 데이터 시각화를 위한 seaborn
import matplotlib.pyplot as plt # 데이터 시각화를 위한 matplotlib
import scipy as sp
from scipy.spatial import distance
sb.set_palette("Set2")

In [2]:
from google.cloud import bigquery
import pandas as pd

# BigQuery 클라이언트 초기화
client = bigquery.Client(project='jnu-idv-21')

# 쿼리 작성
query = """
SELECT *
FROM `0808.baby_food_tbl2`
"""

# 쿼리 실행 및 결과를 Pandas DataFrame으로 변환
df = client.query(query).to_dataframe()
df

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7


### 특정 영양소를 통해 군집화

In [3]:
# 목적 : 밥,죽(탄수화물)/고기 채소/과일
# 영양소를 이용해 특정 음식을 표적
# 사용할 영양소 = 탄수화물, 단백질, 지방, 섬유질(fiber), vitamin_A, thiamine, riboflavin, niacin, vitamin_C

In [8]:
data = df[['carbohydrate','protein','fat','fiber','vitamin_A','thiamine','riboflavin','niacin','vitamin_C']]
data

,carbohydrate,protein,fat,fiber,vitamin_A,thiamine,riboflavin,niacin,vitamin_C
0,18.700001,1.7,0.1,0.1,0.000000,0.0,0.0,0.4,0.0
1,21.400000,6.2,0.4,0.7,4.800000,0.0,0.0,2.7,5.2
2,16.900000,6.1,0.4,0.8,252.399994,0.0,0.0,2.7,1.4
3,15.700000,6.1,0.4,0.7,52.799999,0.0,0.0,2.6,7.4
4,20.700001,2.0,0.1,0.7,125.699997,0.0,0.0,0.5,5.4
...,...,...,...,...,...,...,...,...,...
143,4.200000,11.7,6.0,0.8,241.699997,0.1,0.2,3.5,7.9
144,23.700001,8.8,3.2,0.2,87.599998,0.1,0.2,3.1,7.0
145,27.500000,9.2,0.7,3.3,15.600000,0.1,0.2,2.2,5.2
146,11.200000,10.5,4.6,1.5,221.600006,0.2,0.2,7.5,5.7


In [10]:
# KMeans 군집화 모델 생성
num_clusters = 3  # 군집 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(data)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=42)

In [55]:
# 군집 결과를 데이터프레임에 추가
df['cluster'] = kmeans.labels_
df0 = df
df0

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,cluster
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0,1
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,1
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,2
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,1
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9,2
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,0
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,1
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7,2


In [57]:
# 군집화 확인 : cluster0

cluster0 = 0
cluster0_num = (df0['cluster']==0).sum()
print(f'cluster0에 들어있는 음식 종류 수:{cluster0_num}')
cluster0_food = ','.join(df0.loc[df0['cluster'] == 0, 'food_name'])
print(f'cluster0에 들어있는 음식 종류:{cluster0_food}')

cluster0에 들어있는 음식 종류 수:31
cluster0에 들어있는 음식 종류:찹쌀컬리플라워당근죽,브로콜리당근죽,고구마야채무른밥,옥수수청경채무른밥,청경채죽,단호박참깨버무리,닭고기완자핑거푸드,표고버섯달걀노른자죽,치즈야채무른밥,감자야채달걀노른자찜,달걀노른자고구마죽,달걀노른자비트죽,김노른자무른밥,대구살시금치죽,브로콜리죽,단호박메시,달걀노른자묽은죽,달걀노른자죽,명태옥수수달걀진밥,닭가슴살요구르트샐러드,무숙주비빔밥,시금치죽,잔치국수,고구마경단,쇠고기완자핑거푸드,쇠고기청경채죽,밥경단,카레라이스,쇠고기난자완스,닭고기브로콜리수프,닭죽


In [58]:
# 군집화 확인 : cluster1

cluster1 = 1
cluster1_num = (df0['cluster']==1).sum()
print(f'cluster1에 들어있는 음식 종류 수:{cluster1_num}')
cluster1_food = ','.join(df0.loc[df0['cluster'] == 1, 'food_name'])
print(f'cluster1에 들어있는 음식 종류:{cluster1_food}')

cluster1에 들어있는 음식 종류 수:99
cluster1에 들어있는 음식 종류:찹쌀미음,닭가슴살고구마죽,닭가슴살청경채죽,고구마찹쌀진밥,멜론감자수프,애호박완두콩무른밥,감자바나나요구르트으깸,표고버섯무른밥,콩나물무른밥,두부닭죽,검은콩닭죽,연두부파무른밥,기장미음,청경채미음,단호박죽,양배추죽,쌀죽,오이죽,사과퓨레,애호박죽,대구살죽,호박죽,고구마사과조림,배추연두부죽,얼갈이버섯죽,바나나퓨레,쇠고기두부죽,옥수수미음,가지닭살무른밥,파프리카감자진밥,두부참깨가루죽,오렌지퓨레,감자미음,고구마미음,사과미음,쌀미음,애호박미음,흑미미음,단호박미음,닭고기미음,바나나미음,배미음,수박미음,수수미음,양배추미음,오이미음,과일요구르트범벅,배퓨레,브로콜리미음,쇠고기미음,감자죽,현미묽은죽,현미애호박죽,흑미무죽,닭고기옥수수스프,감자애호박무른밥,단호박고구마진밥,명태죽,쇠고기죽,단호박찹쌀무른밥,흑미두부무른밥,두부죽,고구마두부죽,아기약밥,찹쌀검정콩죽,쇠고기미역무른밥,들깨죽,들깨야채무른밥,브로콜리양송이죽,비트사과치즈진밥,명태옥수수진밥,사과치즈진밥,쇠고기브로콜리죽,쇠고기버섯진밥,완두콩미음,오이치즈죽,조기양송이죽,닭가슴살사과무른밥,조기배추진밥,사과배푸딩,고구마사과메시,닭타락죽,닭가슴살버섯죽,콩나물표고버섯무른밥,비트치즈진밥,감자사과메시,아가용치즈,명태배진밥,애호박쇠고기죽,오이쇠고기감자죽,브로콜리닭무른밥,버섯덮밥,쇠고기연근무른밥,쇠고기파래진밥,쇠고기양송이무른밥,흑미버섯무른밥,감자수제비,양송이버섯감자수프,도미살무른밥


In [59]:
# 군집화 확인 : cluster2

cluster2 = 2
cluster2_num = (df0['cluster']==2).sum()
print(f'cluster2에 들어있는 음식 종류 수:{cluster2_num}')
cluster2_food = ','.join(df0.loc[df0['cluster'] == 2, 'food_name'])
print(f'cluster2에 들어있는 음식 종류:{cluster2_food}')

cluster2에 들어있는 음식 종류 수:18
cluster2에 들어있는 음식 종류:닭가슴살당근죽,당근퓨레,당근죽,가지나물들깨진밥,당근두부죽,현미당근묽은진죽,쇠고기아욱죽,명태배당근진밥,명태당근진밥,감자완자핑거푸드,우엉오므라이스,달걀콩죽,두부달걀노른자찜,노른자영양무른밥,연두부아욱진밥,닭고기시금치그라탕,달걀콩닭고기수프,미역무른밥


In [60]:
# 각 군집별 특징 확인
for cluster_id in range(num_clusters):
    cluster_data = df0[df0['cluster'] == cluster_id]
    cluster_mean = cluster_data.mean()
    print(f"Cluster {cluster_id} 특징:")
    print(cluster_mean[['carbohydrate', 'protein', 'fat', 'fiber', 'vitamin_A', 'thiamine', 'riboflavin', 'niacin', 'vitamin_C']])
    print("-----")

Cluster 0 특징:
carbohydrate     15.532258
protein           5.035484
fat               5.293548
fiber             0.967742
vitamin_A       112.464515
thiamine          0.074194
riboflavin        0.090323
niacin            1.129032
vitamin_C         7.932258
dtype: float64
-----
Cluster 1 특징:
carbohydrate    16.950505
protein          3.596970
fat              2.341414
fiber            0.893939
vitamin_A       12.634343
thiamine         0.035354
riboflavin       0.038384
niacin           1.057576
vitamin_C        4.655556
dtype: float64
-----
Cluster 2 특징:
carbohydrate     18.111111
protein           5.333333
fat               3.333333
fiber             1.772222
vitamin_A       266.272219
thiamine          0.083333
riboflavin        0.083333
niacin            1.677778
vitamin_C         4.611111
dtype: float64
-----


/var/tmp/ipykernel_660033/463850185.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cluster_mean = cluster_data.mean()
/var/tmp/ipykernel_660033/463850185.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cluster_mean = cluster_data.mean()
/var/tmp/ipykernel_660033/463850185.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_on

In [61]:
# vitamin_A 제외

data1 = df[['carbohydrate','protein','fat','fiber','thiamine','riboflavin','niacin','vitamin_C']]
data1

,carbohydrate,protein,fat,fiber,thiamine,riboflavin,niacin,vitamin_C
0,18.700001,1.7,0.1,0.1,0.0,0.0,0.4,0.0
1,21.400000,6.2,0.4,0.7,0.0,0.0,2.7,5.2
2,16.900000,6.1,0.4,0.8,0.0,0.0,2.7,1.4
3,15.700000,6.1,0.4,0.7,0.0,0.0,2.6,7.4
4,20.700001,2.0,0.1,0.7,0.0,0.0,0.5,5.4
...,...,...,...,...,...,...,...,...
143,4.200000,11.7,6.0,0.8,0.1,0.2,3.5,7.9
144,23.700001,8.8,3.2,0.2,0.1,0.2,3.1,7.0
145,27.500000,9.2,0.7,3.3,0.1,0.2,2.2,5.2
146,11.200000,10.5,4.6,1.5,0.2,0.2,7.5,5.7


In [62]:
# KMeans 군집화 모델 생성
num_clusters = 3  # 군집 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(data1)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=42)

In [63]:
# 군집 결과를 데이터프레임에 추가
df['cluster'] = kmeans.labels_
df1 = df
df1

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,cluster
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0,1
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,2
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,1
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,1
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9,1
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,2
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,2
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7,1


In [64]:
# 군집화 확인 : cluster0

cluster0 = 0
cluster0_num = (df1['cluster']==0).sum()
print(f'cluster0에 들어있는 음식 종류 수:{cluster0_num}')
cluster0_food = ','.join(df1.loc[df1['cluster'] == 0, 'food_name'])
print(f'cluster0에 들어있는 음식 종류:{cluster0_food}')

cluster0에 들어있는 음식 종류 수:21
cluster0에 들어있는 음식 종류:멜론감자수프,브로콜리당근죽,고구마야채무른밥,옥수수청경채무른밥,파프리카감자진밥,오렌지퓨레,브로콜리미음,감자애호박무른밥,단호박참깨버무리,브로콜리양송이죽,쇠고기브로콜리죽,고구마사과메시,브로콜리죽,브로콜리닭무른밥,감자완자핑거푸드,시금치죽,쇠고기연근무른밥,감자수제비,쇠고기청경채죽,쇠고기난자완스,닭고기브로콜리수프


In [65]:
# 군집화 확인 : cluster1

cluster1 = 1
cluster1_num = (df1['cluster']==1).sum()
print(f'cluster1에 들어있는 음식 종류 수:{cluster1_num}')
cluster1_food = ','.join(df1.loc[df1['cluster'] == 1, 'food_name'])
print(f'cluster1에 들어있는 음식 종류:{cluster1_food}')

cluster1에 들어있는 음식 종류 수:70
cluster1에 들어있는 음식 종류:찹쌀미음,닭가슴살당근죽,닭가슴살청경채죽,감자바나나요구르트으깸,두부닭죽,기장미음,청경채미음,단호박죽,양배추죽,쌀죽,오이죽,사과퓨레,애호박죽,대구살죽,호박죽,배추연두부죽,얼갈이버섯죽,당근퓨레,감자미음,고구마미음,사과미음,쌀미음,애호박미음,흑미미음,단호박미음,닭고기미음,바나나미음,배미음,수박미음,수수미음,양배추미음,오이미음,과일요구르트범벅,배퓨레,쇠고기미음,감자죽,청경채죽,현미묽은죽,현미애호박죽,흑미무죽,닭고기옥수수스프,당근죽,명태죽,두부죽,당근두부죽,닭고기완자핑거푸드,명태옥수수진밥,표고버섯달걀노른자죽,완두콩미음,오이치즈죽,조기양송이죽,사과배푸딩,감자야채달걀노른자찜,달걀노른자비트죽,대구살시금치죽,닭가슴살버섯죽,단호박메시,감자사과메시,달걀노른자묽은죽,달걀노른자죽,아가용치즈,닭가슴살요구르트샐러드,명태배진밥,무숙주비빔밥,두부달걀노른자찜,고구마경단,쇠고기완자핑거푸드,양송이버섯감자수프,닭고기시금치그라탕,달걀콩닭고기수프


In [67]:
# 군집화 확인 : cluster1

cluster1 = 1
cluster1_num = (df1['cluster']==2).sum()
print(f'cluster1에 들어있는 음식 종류 수:{cluster1_num}')
cluster1_food = ','.join(df1.loc[df1['cluster'] == 2, 'food_name'])
print(f'cluster1에 들어있는 음식 종류:{cluster1_food}')

cluster1에 들어있는 음식 종류 수:57
cluster1에 들어있는 음식 종류:닭가슴살고구마죽,찹쌀컬리플라워당근죽,고구마찹쌀진밥,애호박완두콩무른밥,표고버섯무른밥,콩나물무른밥,검은콩닭죽,연두부파무른밥,고구마사과조림,바나나퓨레,쇠고기두부죽,옥수수미음,가지닭살무른밥,두부참깨가루죽,단호박고구마진밥,쇠고기죽,단호박찹쌀무른밥,흑미두부무른밥,고구마두부죽,아기약밥,찹쌀검정콩죽,가지나물들깨진밥,현미당근묽은진죽,쇠고기미역무른밥,들깨죽,들깨야채무른밥,비트사과치즈진밥,사과치즈진밥,쇠고기아욱죽,쇠고기버섯진밥,닭가슴살사과무른밥,조기배추진밥,치즈야채무른밥,달걀노른자고구마죽,닭타락죽,김노른자무른밥,명태배당근진밥,콩나물표고버섯무른밥,비트치즈진밥,명태당근진밥,명태옥수수달걀진밥,애호박쇠고기죽,오이쇠고기감자죽,버섯덮밥,우엉오므라이스,달걀콩죽,노른자영양무른밥,잔치국수,쇠고기파래진밥,쇠고기양송이무른밥,흑미버섯무른밥,연두부아욱진밥,밥경단,카레라이스,닭죽,도미살무른밥,미역무른밥


In [68]:
# 각 군집별 특징 확인
for cluster_id in range(num_clusters):
    cluster_data = df1[df1['cluster'] == cluster_id]
    cluster_mean = cluster_data.mean()
    print(f"Cluster {cluster_id} 특징:")
    print(cluster_mean[['carbohydrate', 'protein', 'fat', 'fiber', 'vitamin_A', 'thiamine', 'riboflavin', 'niacin', 'vitamin_C']])
    print("-----")

Cluster 0 특징:
carbohydrate    15.485714
protein          3.933333
fat              3.952381
fiber            0.980952
vitamin_A       69.180953
thiamine         0.066667
riboflavin       0.066667
niacin           1.033333
vitamin_C       16.600000
dtype: float64
-----
Cluster 1 특징:
carbohydrate    12.532857
protein          3.141429
fat              1.585714
fiber            0.548571
vitamin_A       55.767142
thiamine         0.030000
riboflavin       0.040000
niacin           0.938571
vitamin_C        3.290000
dtype: float64
-----
Cluster 2 특징:
carbohydrate    22.510526
protein          5.363158
fat              4.594737
fiber            1.603509
vitamin_A       73.221052
thiamine         0.066667
riboflavin       0.068421
niacin           1.447368
vitamin_C        3.700000
dtype: float64
-----


/var/tmp/ipykernel_660033/3636704174.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cluster_mean = cluster_data.mean()
/var/tmp/ipykernel_660033/3636704174.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cluster_mean = cluster_data.mean()
/var/tmp/ipykernel_660033/3636704174.py:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric

In [69]:
# vitamin_A로만 군집화

data2 = df[['vitamin_A']]
data2

,vitamin_A
0,0.000000
1,4.800000
2,252.399994
3,52.799999
4,125.699997
...,...
143,241.699997
144,87.599998
145,15.600000
146,221.600006


In [70]:
# KMeans 군집화 모델 생성
num_clusters = 3  # 군집 개수 설정
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(data2)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=3, random_state=42)

In [71]:
# 군집 결과를 데이터프레임에 추가
df['cluster'] = kmeans.labels_
df2 = df
df2

,food_name,ingredient_name,ingredient_index,ingredient_gram,calorie,carbohydrate,protein,fat,fiber,calcium,phosphorus,iron,sodium,potassium,vitamin_A,thiamine,riboflavin,niacin,vitamin_C,cluster
0,찹쌀미음,"[쌀, 찹쌀, 백미]","[0, 18, 19]","[-1, -1, 23]",86.000000,18.700001,1.7,0.1,0.1,0.900000,34.700001,0.5,0.700000,43.900002,0.000000,0.0,0.0,0.4,0.0,1
1,닭가슴살고구마죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 고구마...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 8, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",117.000000,21.400000,6.2,0.4,0.7,9.800000,63.400002,0.5,16.400000,157.199997,4.800000,0.0,0.0,2.7,5.2,1
2,닭가슴살당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 당근,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 20]",98.199997,16.900000,6.1,0.4,0.8,12.600000,60.000000,0.5,19.200001,143.800003,252.399994,0.0,0.0,2.7,1.4,2
3,닭가슴살청경채죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 청경채...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 39]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, 15]",93.500000,15.700000,6.1,0.4,0.7,18.500000,58.299999,0.5,15.600000,105.400002,52.799999,0.0,0.0,2.6,7.4,1
4,찹쌀컬리플라워당근죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 쌀, 찹쌀, 백미, 컬리플라워,...","[0, 1, 2, 3, 4, 5, 0, 18, 19, 67, 47, 7]","[-1, -1, -1, -1, -1, 20, -1, -1, 5, 10, -1, 10]",94.199997,20.700001,2.0,0.1,0.7,9.000000,29.000000,0.5,6.400000,96.000000,125.699997,0.0,0.0,0.5,5.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,닭고기시금치그라탕,"[닭고기, 가슴살, 날것, 시금치, 생것, 노지, 당근, 생것, 양파, 생것, 국내...","[27, 62, 28, 49, 7, 50, 47, 7, 66, 7, 31, 98, ...","[-1, -1, 30, -1, -1, 10, -1, 10, -1, -1, 10, -...",117.599998,4.200000,11.7,6.0,0.8,133.199997,256.600006,0.6,265.100006,223.399994,241.699997,0.1,0.2,3.5,7.9,2
144,닭죽,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 닭고기, 가슴살, 날것, 분유,...","[0, 1, 2, 3, 4, 5, 27, 62, 28, 63, 64]","[-1, -1, -1, -1, -1, 20, -1, -1, 20, -1, 15]",162.000000,23.700001,8.8,3.2,0.2,97.599998,123.099998,1.4,35.200001,144.300003,87.599998,0.1,0.2,3.1,7.0,0
145,도미살무른밥,"[쌀, 멥쌀, 논벼, 백미 국내산, 일반형, 일품, 돔, 감성돔, 배추, 생것, 양...","[0, 1, 2, 3, 4, 5, 83, 84, 40, 7, 73, 7, 52, 53]","[-1, -1, -1, -1, -1, 30, -1, 30, -1, 10, -1, 1...",155.000000,27.500000,9.2,0.7,3.3,89.000000,128.800003,1.4,338.700012,982.599976,15.600000,0.1,0.2,2.2,5.2,1
146,달걀콩닭고기수프,"[계란, 전란, 생것, 닭고기, 가슴살, 날것, 당근, 생것, 완두콩, 생것, 옥수...","[41, 61, 7, 27, 62, 28, 47, 7, 17, 7, 15, 16, ...","[-1, -1, 20, -1, -1, 20, -1, 10, -1, 10, -1, -...",129.899994,11.200000,10.5,4.6,1.5,84.300003,145.100006,1.6,55.900002,228.100006,221.600006,0.2,0.2,7.5,5.7,2


In [72]:
# 군집화 확인 : cluster0

cluster0 = 0
cluster0_num = (df2['cluster']==0).sum()
print(f'cluster0에 들어있는 음식 종류 수:{cluster0_num}')
cluster0_food = ','.join(df.loc[df2['cluster'] == 0, 'food_name'])
print(f'cluster0에 들어있는 음식 종류:{cluster0_food}')

cluster0에 들어있는 음식 종류 수:31
cluster0에 들어있는 음식 종류:찹쌀컬리플라워당근죽,브로콜리당근죽,고구마야채무른밥,옥수수청경채무른밥,청경채죽,단호박참깨버무리,닭고기완자핑거푸드,표고버섯달걀노른자죽,치즈야채무른밥,감자야채달걀노른자찜,달걀노른자고구마죽,달걀노른자비트죽,김노른자무른밥,대구살시금치죽,브로콜리죽,단호박메시,달걀노른자묽은죽,달걀노른자죽,명태옥수수달걀진밥,닭가슴살요구르트샐러드,무숙주비빔밥,시금치죽,잔치국수,고구마경단,쇠고기완자핑거푸드,쇠고기청경채죽,밥경단,카레라이스,쇠고기난자완스,닭고기브로콜리수프,닭죽


In [52]:
# 군집화 확인 : cluster1

cluster1 = 1
cluster1_num = (df2['cluster']==1).sum()
print(f'cluster1에 들어있는 음식 종류 수:{cluster1_num}')
cluster1_food = ','.join(df.loc[df2['cluster'] == 1, 'food_name'])
print(f'cluster1에 들어있는 음식 종류:{cluster1_food}')

cluster1에 들어있는 음식 종류 수:99
cluster1에 들어있는 음식 종류:찹쌀미음,닭가슴살고구마죽,닭가슴살청경채죽,고구마찹쌀진밥,멜론감자수프,애호박완두콩무른밥,감자바나나요구르트으깸,표고버섯무른밥,콩나물무른밥,두부닭죽,검은콩닭죽,연두부파무른밥,기장미음,청경채미음,단호박죽,양배추죽,쌀죽,오이죽,사과퓨레,애호박죽,대구살죽,호박죽,고구마사과조림,배추연두부죽,얼갈이버섯죽,바나나퓨레,쇠고기두부죽,옥수수미음,가지닭살무른밥,파프리카감자진밥,두부참깨가루죽,오렌지퓨레,감자미음,고구마미음,사과미음,쌀미음,애호박미음,흑미미음,단호박미음,닭고기미음,바나나미음,배미음,수박미음,수수미음,양배추미음,오이미음,과일요구르트범벅,배퓨레,브로콜리미음,쇠고기미음,감자죽,현미묽은죽,현미애호박죽,흑미무죽,닭고기옥수수스프,감자애호박무른밥,단호박고구마진밥,명태죽,쇠고기죽,단호박찹쌀무른밥,흑미두부무른밥,두부죽,고구마두부죽,아기약밥,찹쌀검정콩죽,쇠고기미역무른밥,들깨죽,들깨야채무른밥,브로콜리양송이죽,비트사과치즈진밥,명태옥수수진밥,사과치즈진밥,쇠고기브로콜리죽,쇠고기버섯진밥,완두콩미음,오이치즈죽,조기양송이죽,닭가슴살사과무른밥,조기배추진밥,사과배푸딩,고구마사과메시,닭타락죽,닭가슴살버섯죽,콩나물표고버섯무른밥,비트치즈진밥,감자사과메시,아가용치즈,명태배진밥,애호박쇠고기죽,오이쇠고기감자죽,브로콜리닭무른밥,버섯덮밥,쇠고기연근무른밥,쇠고기파래진밥,쇠고기양송이무른밥,흑미버섯무른밥,감자수제비,양송이버섯감자수프,도미살무른밥


In [54]:
# 군집화 확인 : cluster2

cluster2 = 2
cluster2_num = (df2['cluster']==2).sum()
print(f'cluster2에 들어있는 음식 종류 수:{cluster2_num}')
cluster2_food = ','.join(df.loc[df2['cluster'] == 2, 'food_name'])
print(f'cluster2에 들어있는 음식 종류:{cluster2_food}')

cluster2에 들어있는 음식 종류 수:18
cluster2에 들어있는 음식 종류:닭가슴살당근죽,당근퓨레,당근죽,가지나물들깨진밥,당근두부죽,현미당근묽은진죽,쇠고기아욱죽,명태배당근진밥,명태당근진밥,감자완자핑거푸드,우엉오므라이스,달걀콩죽,두부달걀노른자찜,노른자영양무른밥,연두부아욱진밥,닭고기시금치그라탕,달걀콩닭고기수프,미역무른밥


In [73]:
# 결론
## 첫번째 데이터와 세번째 데이터 군집화 결과 같음
## 분류 기준 = vitamin_A      

# 각 군집별 특징 확인
for cluster_id in range(num_clusters):
    cluster_data = df2[df2['cluster'] == cluster_id]
    cluster_mean = cluster_data.mean()
    print(f"Cluster {cluster_id} 특징:")
    print(cluster_mean[['carbohydrate', 'protein', 'fat', 'fiber', 'vitamin_A', 'thiamine', 'riboflavin', 'niacin', 'vitamin_C']])
    print("-----")

Cluster 0 특징:
carbohydrate     15.532258
protein           5.035484
fat               5.293548
fiber             0.967742
vitamin_A       112.464515
thiamine          0.074194
riboflavin        0.090323
niacin            1.129032
vitamin_C         7.932258
dtype: float64
-----
Cluster 1 특징:
carbohydrate    16.950505
protein          3.596970
fat              2.341414
fiber            0.893939
vitamin_A       12.634343
thiamine         0.035354
riboflavin       0.038384
niacin           1.057576
vitamin_C        4.655556
dtype: float64
-----
Cluster 2 특징:
carbohydrate     18.111111
protein           5.333333
fat               3.333333
fiber             1.772222
vitamin_A       266.272219
thiamine          0.083333
riboflavin        0.083333
niacin            1.677778
vitamin_C         4.611111
dtype: float64
-----


/var/tmp/ipykernel_660033/2308505550.py:8: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cluster_mean = cluster_data.mean()
/var/tmp/ipykernel_660033/2308505550.py:8: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  cluster_mean = cluster_data.mean()
/var/tmp/ipykernel_660033/2308505550.py:8: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric